# Project 3: Data and Maps!

Thanks to John P. Dickersion for the project idea!

**Posted:** Nov 7th, 2019.

**Due:** Nov. 26th, 2019.

In this project we are going to work with a fairly clean set of data from Baltimore crime data covering the years 2011 and 2012.  This is a fairly open ended project, you will need to work with the data a bit and come up with your own things to show.

In [30]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd
# Load Stats
from scipy import stats
# import folium TODO: add folium to path
import re

import datetime as dt

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
matplotlib.style.use('fivethirtyeight')

## Part 1: Data Wrangling.

The data is a bit messy to start out with.  Perform the following tasks to make it clean and tidy.

1. Split the `Location 1` column into a `lat` and `long` columns.  Ensure that the columns are of float type and you drop any record that is missing a location.
2. You can drop the `arrest`, `post`, `charge`, and the `Location 1` column.
3. Merge the date and time column and make sure they are the proper type.  Drop any row that does not have a date and time.
4. Set the index so that we can sort and slice based on the date/time.
5. Drop any records that have NA values.
6. Go through the remaining columns and ensure you have set the dtype properly.
7. Display the head of the table and the dtypes in your notebook.





### Code Description:
First I dropped the na values using dropna
To create latitude and longituted columns, I defined functions to split and clean up the `Location 1` entries. Then I used .apply() to create new lat and long columns.

To merge datetimes into one column, I created a for loop to split date and time strings then create datetime objects with them. The for loop created entries of a new column in the dataframe.

I then set the index as the datetime columns and dropped unneccessary columns.

In [27]:
#Read in dataframe
raw_df = pd.read_csv("./BPD_Arrests.csv")

#drop NaN location values
raw_df.dropna(axis = 0, inplace = True)

#define function to return latitude (first value)
def get_lat(lat_long):
    temp_array = re.split(" ", str(lat_long))
    return float(re.sub(r"\(|\,", "", temp_array[0]))

#define function to get longitute (second value)
def get_long(lat_long):
    temp_array = re.split(" ", str(lat_long))
    return float(re.sub(r"\)", "", temp_array[1]))

# make colums for latitude and longitude
raw_df["Latitude"] = raw_df["Location 1"].apply(get_lat)
raw_df["Longitude"] = raw_df["Location 1"].apply(get_long)

# Drop the arrest, post, charge, and the Location 1 column
raw_df.drop(axis = 1, columns = ["arrest", "post", "charge", "Location 1"], inplace = True)

,age,race,sex,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,chargeDescription,district,neighborhood,Latitude,Longitude
1,37,B,M,01/01/2011,00:01:00,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,Reckless Endangerment || Hand Gun Violation,SOUTHERN,Carrollton Ridge,39.281403,-76.648364
3,50,B,M,01/01/2011,00:04:00,2100 Ashburton St,79-Other,2100 Ashburton St,Reg Firearm:Illegal Possession || Hgv,WESTERN,Panway/Braddish Avenue,39.311720,-76.662355
5,41,B,M,01/01/2011,00:05:00,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,Reckless Endangerment || Handgun Violation,SOUTHERN,Cherry Hill,39.244989,-76.627358
6,29,B,M,01/01/2011,00:05:00,800 N Monroe St,79-Other,800 N Monroe St,Handgun On Person || Handgun Violation,WESTERN,Midtown-Edmondson,39.297982,-76.647511
9,53,B,M,01/01/2011,00:15:00,3300 Woodland Ave,54-Armed Person,3300 Woodland Av,Reckless Endangerment || Hgv,NORTHWESTERN,Central Park Heights,39.343677,-76.672730


In [37]:
# create a new columns using the arrays to initialize datetime objects
for index, row in raw_df.iterrows():
    #split date and time into arrays
    date_array = re.split("/", row["arrestDate"])
    time_array = re.split(":", row["arrestTime"])
    #make a column with datetime object
    raw_df['arrestDateTime'] = dt.datetime(
        int(date_array[2]), 
        int(date_array[0]), 
        int(date_array[1]), 
        int(time_array[0]), 
        int(time_array[1]), 
        int(time_array[2])
    )

# drop arrestTime and arrestDate columns
raw_df.drop(axis = 1, columns = ["arrestTime", "arrestDate"], inplace = True)


,age,race,sex,arrestLocation,incidentOffense,incidentLocation,chargeDescription,district,neighborhood,Latitude,Longitude,arrestDateTime
1,37,B,M,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,Reckless Endangerment || Hand Gun Violation,SOUTHERN,Carrollton Ridge,39.281403,-76.648364,2012-12-31 23:55:00
3,50,B,M,2100 Ashburton St,79-Other,2100 Ashburton St,Reg Firearm:Illegal Possession || Hgv,WESTERN,Panway/Braddish Avenue,39.311720,-76.662355,2012-12-31 23:55:00
5,41,B,M,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,Reckless Endangerment || Handgun Violation,SOUTHERN,Cherry Hill,39.244989,-76.627358,2012-12-31 23:55:00
6,29,B,M,800 N Monroe St,79-Other,800 N Monroe St,Handgun On Person || Handgun Violation,WESTERN,Midtown-Edmondson,39.297982,-76.647511,2012-12-31 23:55:00
9,53,B,M,3300 Woodland Ave,54-Armed Person,3300 Woodland Av,Reckless Endangerment || Hgv,NORTHWESTERN,Central Park Heights,39.343677,-76.672730,2012-12-31 23:55:00


In [41]:
#set index as datetimes
raw_df.set_index(keys = "arrestDateTime")

# check that all columns have the correct dtype
raw_df.dtypes

age                           int64
race                         object
sex                          object
arrestLocation               object
incidentOffense              object
incidentLocation             object
chargeDescription            object
district                     object
neighborhood                 object
Latitude                    float64
Longitude                   float64
arrestDateTime       datetime64[ns]
dtype: object

In [47]:
# display dataframe
raw_df.head()

,age,race,sex,arrestLocation,incidentOffense,incidentLocation,chargeDescription,district,neighborhood,Latitude,Longitude,arrestDateTime
1,37,B,M,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,Reckless Endangerment || Hand Gun Violation,SOUTHERN,Carrollton Ridge,39.281403,-76.648364,2012-12-31 23:55:00
3,50,B,M,2100 Ashburton St,79-Other,2100 Ashburton St,Reg Firearm:Illegal Possession || Hgv,WESTERN,Panway/Braddish Avenue,39.311720,-76.662355,2012-12-31 23:55:00
5,41,B,M,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,Reckless Endangerment || Handgun Violation,SOUTHERN,Cherry Hill,39.244989,-76.627358,2012-12-31 23:55:00
6,29,B,M,800 N Monroe St,79-Other,800 N Monroe St,Handgun On Person || Handgun Violation,WESTERN,Midtown-Edmondson,39.297982,-76.647511,2012-12-31 23:55:00
9,53,B,M,3300 Woodland Ave,54-Armed Person,3300 Woodland Av,Reckless Endangerment || Hgv,NORTHWESTERN,Central Park Heights,39.343677,-76.672730,2012-12-31 23:55:00


### Question 1:
How many records did we drop using our processing above?  Do you think this will affect our data later?  What type of missingness do you think these values have? 

In [60]:
# read in the data again and check the columns
initial_df = pd.read_csv("./BPD_Arrests.csv")

initial_df.dtypes
# initial_df[initial_df["chargeDescription"] == "Violation Of Probation || Violation Of Probation"]["Location 1"].value_counts()

arrest               float64
age                    int64
race                  object
sex                   object
arrestDate            object
arrestTime            object
arrestLocation        object
incidentOffense       object
incidentLocation      object
charge                object
chargeDescription     object
district              object
post                 float64
neighborhood          object
Location 1            object
dtype: object

### Answer 1:
The initial data contained 104528 rows. After we dropped the rows with na values, the dataframe contained 54040 rows. We dropped a little less than half our data. This can lead to our sample having a **larger variance** and/or having a **biased sample.**

### Question 2:
Thinking about the kinds of missing-ness in our data.  What is one imputation method that we could have used to fill in some gaps?  Implement one such method that is not just `dropna`.

### Answer 2:
I infer that the data is *Missing at Random*. In order to account for the missing data. I will use other information to predict missing values. When the lat/long of a row is missing, I will check if the neighboorhood, arrest location, and incident location are blank. If they are, I will discard the entry, because it tells us very little about the arrest. If any of these values are known, I will fill our missing data with a random sample with the same values. This is a combination of bootstrapping and value prediction.

In [65]:
# define a method to add a new row to the dataframe with desired values.
# def add_row(dataframe):
#     row = dataframe.iloc[0]
#     row
for j, row in initial_df.iterrows():
    if (pd.isna(row["Location 1"])):
        # initialize empty strings
        d = ""
        n = ""
        a = ""
        i = ""
        # get information we will use to predict the data
        if (!pd.isna(row["district"])):
            d = row["district"]
        if (!pd.isna(row["neighborhood"])):
            n = row["neighborhood"]
        if (!pd.isna(row["arrestLocation"])):
            a = row["arrestLocation"]
        if (!pd.isna(row["incidentLocation"])):
            i = row["incidentLocation"]
        
        # bootstrap the missing data using known data from rows with like attributes.
        if (d or n or a or i):
            matching_sample = raw_df[raw_df["district" == d] &&
                                    raw_df["neighborhood" == n] &&
                                    raw_df["arrestLocation" == a] &&
                                    raw_df["incidentLocation" == i]]
# initial_df.iloc[0]

0
8
22
34
35
43
44
47
48
49
51
54
58
61
62
64
69
71
74
77
84
85
86
90
93
94
95
97
99
100
102
103
105
108
109
113
114
115
117
119
122
125
126
130
138
140
142
145
150
151
152
153
154
156
157
158
159
160
161
162
163
164
165
166
167
168
170
171
172
173
174
175
176
177
178
179
180
181
182
184
186
187
189
195
196
197
198
199
203
207
209
211
224
226
229
235
239
244
245
246
248
251
253
254
256
257
258
259
260
266
267
268
269
270
271
273
274
275
276
277
278
279
280
281
284
285
291
296
298
305
306
314
315
316
324
327
328
331
332
333
334
335
338
339
340
341
342
343
347
348
353
354
356
357
358
359
370
371
372
373
374
375
377
380
381
382
383
384
385
386
387
390
395
396
405
408
410
411
412
416
422
431
432
433
434
435
436
437
438
439
440
441
443
444
446
447
451
452
453
457
458
459
461
462
463
464
469
474
475
476
478
480
481
483
484
490
491
492
494
500
501
502
503
508
509
518
521
525
526
527
528
529
530
531
537
541
545
546
548
554
556
559
564
567
569
570
572
573
574
575
576
577
578
579
581
582
583
584

4959
4963
4964
4965
4966
4969
4971
4972
4973
4974
4975
4976
4977
4978
4980
4981
4986
4988
4989
4990
4993
4994
4995
4996
4997
4998
4999
5008
5010
5014
5020
5021
5024
5025
5026
5027
5031
5032
5033
5034
5035
5036
5038
5039
5041
5043
5046
5047
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5064
5065
5066
5068
5069
5071
5074
5075
5077
5079
5086
5088
5089
5093
5096
5099
5102
5104
5105
5106
5111
5115
5118
5119
5123
5124
5125
5126
5129
5131
5134
5135
5137
5140
5142
5143
5144
5145
5151
5160
5161
5162
5164
5169
5170
5171
5172
5174
5177
5178
5181
5191
5192
5197
5200
5201
5206
5207
5214
5215
5219
5220
5222
5223
5226
5227
5231
5232
5234
5237
5238
5241
5242
5243
5247
5248
5249
5254
5255
5257
5258
5260
5267
5268
5270
5273
5278
5279
5282
5283
5285
5286
5287
5289
5290
5291
5292
5293
5294
5295
5296
5298
5300
5301
5302
5303
5304
5308
5309
5310
5311
5313
5316
5322
5325
5326
5330
5331
5332
5335
5339
5344
5348
5352
5353
5359
5360
5361
5367
5370
5375
5376
5381
5383
5388
5391
5392
5393
5396
5397
5398


9571
9572
9573
9575
9577
9581
9582
9584
9585
9588
9590
9591
9592
9593
9594
9595
9598
9599
9600
9601
9605
9606
9607
9610
9611
9612
9613
9614
9615
9617
9618
9620
9621
9622
9623
9624
9625
9626
9627
9629
9631
9634
9635
9637
9642
9646
9648
9651
9652
9655
9656
9657
9658
9661
9662
9663
9664
9666
9667
9670
9677
9678
9682
9685
9688
9693
9697
9700
9701
9703
9704
9705
9707
9708
9709
9710
9711
9713
9714
9715
9717
9718
9719
9720
9721
9722
9723
9725
9726
9727
9728
9729
9730
9731
9734
9735
9738
9739
9740
9741
9742
9749
9750
9752
9760
9767
9768
9769
9771
9779
9780
9781
9782
9783
9784
9787
9788
9796
9797
9798
9799
9800
9801
9806
9807
9810
9813
9814
9816
9818
9819
9822
9826
9827
9828
9830
9832
9835
9837
9838
9844
9847
9848
9852
9856
9858
9861
9867
9870
9872
9877
9881
9884
9888
9889
9893
9894
9897
9898
9900
9901
9902
9903
9904
9905
9906
9909
9910
9914
9915
9916
9917
9918
9919
9920
9922
9923
9924
9926
9927
9928
9929
9930
9933
9936
9937
9939
9940
9945
9947
9948
9951
9960
9961
9972
9973
9974
9984
9986
9989


12969
12975
12976
12977
12978
12979
12980
12981
12982
12984
12986
12989
12994
12995
12996
12997
12999
13000
13001
13003
13004
13005
13009
13010
13011
13015
13016
13017
13018
13019
13022
13023
13030
13031
13032
13033
13034
13036
13037
13041
13042
13045
13046
13055
13058
13059
13061
13065
13066
13072
13080
13081
13082
13084
13087
13091
13094
13101
13108
13112
13113
13117
13119
13120
13121
13129
13130
13132
13134
13135
13137
13139
13140
13142
13143
13145
13146
13147
13150
13151
13152
13156
13157
13158
13159
13160
13163
13172
13173
13174
13176
13178
13181
13183
13184
13189
13190
13192
13194
13198
13199
13200
13201
13207
13208
13210
13211
13225
13226
13227
13231
13238
13239
13240
13244
13245
13249
13250
13251
13256
13261
13264
13265
13266
13268
13271
13272
13273
13274
13275
13276
13277
13279
13280
13282
13283
13285
13286
13287
13288
13289
13290
13291
13292
13293
13295
13296
13297
13299
13302
13308
13310
13312
13314
13315
13316
13318
13324
13325
13329
13330
13331
13333
13335
13342
13343
1334

16430
16431
16432
16433
16434
16435
16436
16437
16438
16441
16442
16443
16444
16445
16447
16448
16451
16455
16456
16459
16462
16464
16465
16469
16476
16477
16480
16481
16484
16485
16486
16487
16488
16491
16492
16495
16496
16497
16498
16499
16500
16502
16504
16505
16506
16507
16509
16512
16513
16514
16515
16517
16518
16521
16522
16523
16524
16529
16531
16532
16537
16538
16539
16540
16541
16542
16547
16549
16550
16551
16552
16555
16557
16559
16562
16563
16565
16576
16577
16578
16583
16584
16587
16593
16594
16596
16602
16605
16607
16615
16616
16619
16627
16637
16638
16639
16640
16641
16645
16647
16649
16651
16652
16657
16659
16664
16666
16671
16672
16675
16680
16681
16682
16704
16708
16709
16710
16713
16714
16719
16721
16723
16725
16727
16728
16729
16730
16737
16740
16743
16744
16745
16748
16751
16752
16753
16755
16756
16762
16763
16764
16765
16767
16768
16769
16770
16771
16772
16773
16774
16775
16776
16777
16781
16782
16783
16784
16786
16787
16788
16789
16794
16795
16796
16797
16798
1679

...

83721
83722
83723
83728
83731
83732
83733
83738
83744
83745
83746
83747
83750
83751
83755
83757
83761
83762
83765
83768
83769
83771
83774
83775
83779
83785
83788
83790
83791
83792
83793
83795
83796
83801
83803
83804
83810
83813
83817
83818
83819
83820
83821
83822
83823
83824
83825
83827
83831
83836
83841
83842
83845
83847
83848
83850
83853
83854
83857
83866
83868
83869
83873
83874
83876
83877
83878
83879
83880
83883
83884
83885
83886
83890
83896
83899
83901
83902
83903
83904
83909
83910
83912
83917
83925
83934
83936
83937
83941
83942
83945
83947
83952
83954
83955
83964
83968
83970
83972
83973
83974
83975
83976
83977
83979
83980
83981
83984
83985
83986
83988
83989
83990
83991
83992
84002
84003
84004
84008
84010
84011
84012
84014
84017
84018
84024
84025
84034
84038
84039
84041
84043
84047
84050
84052
84060
84061
84062
84063
84068
84082
84083
84084
84085
84089
84090
84091
84097
84104
84116
84118
84123
84128
84132
84142
84147
84151
84154
84156
84157
84160
84169
84172
84175
84176
84177
8418

88042
88044
88048
88049
88050
88051
88052
88053
88058
88060
88061
88062
88063
88066
88067
88068
88069
88071
88074
88078
88079
88082
88083
88084
88088
88094
88099
88106
88108
88110
88113
88116
88117
88120
88124
88132
88138
88140
88142
88148
88149
88150
88153
88154
88156
88160
88172
88173
88174
88179
88181
88191
88192
88194
88195
88196
88197
88199
88200
88202
88203
88205
88206
88209
88210
88211
88213
88215
88217
88218
88220
88221
88222
88223
88224
88225
88228
88229
88230
88232
88235
88237
88240
88242
88243
88246
88247
88248
88249
88251
88252
88253
88255
88261
88262
88266
88267
88275
88279
88280
88281
88284
88287
88288
88291
88295
88307
88309
88314
88331
88333
88340
88356
88357
88358
88368
88369
88370
88371
88372
88373
88374
88375
88378
88379
88380
88381
88382
88383
88384
88385
88386
88387
88388
88390
88391
88396
88397
88398
88399
88403
88404
88405
88408
88412
88413
88414
88415
88416
88419
88420
88421
88429
88438
88439
88441
88444
88445
88447
88448
88450
88452
88454
88456
88457
88458
8846

93039
93040
93041
93045
93053
93054
93055
93058
93061
93071
93076
93086
93094
93096
93106
93107
93108
93110
93111
93115
93116
93120
93122
93125
93127
93129
93132
93133
93135
93136
93138
93145
93147
93148
93149
93151
93152
93154
93155
93164
93177
93180
93191
93192
93193
93196
93202
93213
93214
93215
93222
93230
93233
93234
93237
93238
93248
93254
93257
93260
93262
93264
93266
93267
93268
93271
93275
93278
93282
93286
93294
93297
93299
93303
93304
93308
93313
93316
93317
93319
93320
93325
93326
93329
93331
93332
93333
93334
93335
93336
93337
93342
93343
93344
93345
93346
93347
93348
93351
93353
93358
93359
93360
93361
93362
93363
93364
93365
93369
93372
93373
93374
93376
93377
93379
93380
93381
93382
93386
93387
93391
93393
93394
93403
93406
93412
93414
93416
93424
93425
93432
93433
93434
93437
93443
93448
93450
93453
93454
93455
93456
93457
93460
93462
93464
93465
93466
93469
93470
93472
93474
93475
93476
93479
93482
93483
93484
93487
93488
93491
93492
93493
93496
93497
93499
93502
9350

97095
97096
97099
97101
97104
97107
97108
97109
97113
97116
97123
97124
97127
97128
97134
97136
97137
97140
97142
97149
97150
97151
97152
97153
97154
97163
97178
97179
97180
97182
97185
97187
97190
97200
97201
97202
97207
97208
97209
97211
97212
97221
97224
97226
97227
97228
97230
97236
97239
97240
97244
97248
97249
97250
97255
97258
97270
97271
97273
97275
97276
97283
97285
97289
97291
97292
97293
97294
97295
97296
97297
97298
97302
97303
97304
97305
97306
97307
97308
97309
97310
97311
97312
97314
97315
97316
97317
97318
97319
97320
97322
97328
97329
97337
97338
97339
97341
97344
97347
97348
97349
97352
97355
97359
97363
97364
97365
97368
97369
97370
97371
97372
97377
97380
97382
97388
97390
97395
97396
97398
97399
97401
97402
97406
97423
97426
97427
97429
97433
97435
97447
97460
97466
97467
97476
97483
97484
97485
97486
97488
97489
97491
97492
97494
97495
97496
97499
97500
97502
97503
97507
97508
97510
97512
97513
97516
97517
97519
97527
97529
97530
97532
97533
97535
97537
97539
9754

101630
101631
101632
101633
101634
101635
101636
101637
101638
101641
101642
101643
101644
101645
101647
101650
101651
101654
101657
101660
101661
101662
101668
101669
101670
101672
101673
101675
101678
101679
101683
101686
101688
101690
101693
101694
101702
101705
101706
101718
101722
101726
101730
101733
101734
101745
101746
101748
101762
101763
101764
101765
101766
101767
101771
101772
101775
101780
101781
101782
101789
101790
101791
101792
101793
101795
101796
101797
101798
101799
101800
101801
101802
101803
101804
101805
101808
101814
101817
101820
101823
101824
101825
101831
101834
101835
101837
101840
101841
101842
101843
101845
101848
101852
101863
101867
101868
101870
101875
101879
101882
101889
101893
101899
101901
101905
101906
101910
101913
101917
101919
101922
101923
101924
101925
101926
101927
101928
101939
101948
101951
101952
101954
101956
101957
101958
101959
101960
101962
101963
101965
101966
101968
101971
101974
101977
101980
101982
101987
101991
102001
102010
102014

## Part 2: Exploratory Data Analysis

We can use the Pandas time and date slicing functions to group our data by either day, quarter, or time.  Have a look at [pd.between_time()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.between_time.html).  I want you to explore this data in some interesting ways.

### Problem 1.
Use cut and other Pandas functions to display the joint distribution of Age and Race.  This table should not have every age in it but break the age down into a reasonable number of sub groups.

Pick another pair of variables.  Display a joint or conditional distribution and explain **why** you chose it and what the take away message is.

### Problem 2.

Pick (at least) three nieghborhoods from the data, show the crime in 2011 versus 2012 for each of these neighborhoods on one plot.  Make sure that you use visaul features to distinguish the two years.

**Hint:** You may want to look back at the lab where we worked with baby names... and maybe the [unstack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html) function.

### Problem 3.

Show me one other interesting thing about the data.  It can be anything you find interesting but I'd encourage you to use an advanced method from class (regression, classification, hypothesis testing etc.).  If you can, maybe look at something like [the demographics of Balitmore](https://en.wikipedia.org/wiki/Baltimore) and compare those to what is in our data.



## Part 3: Interactive Maps.

Using the following code stub to start up an interactive map. You can find more information about folium here: https://github.com/python-visualization/folium/ and https://folium.readthedocs.org//


### Problem 5.

Add graphical elements to display the data. For instance, add circles, with colors indicating sex. Or circles with colors indicating race. Or anything else that strikes your fancy.  Plot some colors over the map to illustrate some joint or conditional distribution of the data.

**Explain using Markdown Cells** *what* you have shown in your map, *why* you have shown it in your map, and *how* a user should interpret this information.

In [3]:
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)
map_osm

## Submission

Prepare a Jupyter notebook that includes for each Problem: (a) code to carry out the step discussed, (b) output showing the result of your code, and (c) a short prose description of how your code works. Remember, the writeup you are preparing is intended to communicate your data analysis effectively. Thoughtlessly showing large amounts of output in your writeup defeats that purpose.

All axes in plots should be labeled in an informative manner. Your answers to any question that refers to a plot should include both (a) a text description of your plot, and (b) a sentence or two of interpretation as it relates to the question asked.

Submit this completed notebook which contains your answers as markdown cells to [Canvas](https://tulane.instructure.com/)

## Grading Rubric

Note that code that does not work will not be graded and you will receive a 0 for that section.  We reserve the right to deduct points for things like general sloppiness of the notebook, poor labels, unlabeled axes, etc.  You should include markdown cells to break up your notebook and **clearly label** the problems and questions below.

* Part 0 Professionalism (10 points).
  * You have used both code comments and markdown cells to professionally and clearly document your work including having a clear and clean notebook; linking to resources and documents; and doing so with code that is reasonable and efficient.

* Part 1 Wrangling (20 Points).
  * (10 Points)  Data is loaded correctly and directions are followed for munging the data appropatly.
  * (10 Points) Questions are answered in a reasonable manner.  A suggested way to impute data is present along with code.
* Part 2 Exploratory Data Analysis (40 Points).
  * (20 Points) Problem 1: Distributions are compute correctly, tables are shown, explination is coherent and clear.
  * (10 Points) Problem 2: Graph is present, visual features are present to distingush the required elements.
  * (10 Points) Problem 3: Code is present to compute an interesting feature of the data.  The feature is interpreted in a written markdown cell.
* Part 3 Interactive Maps (30 Points).
  * (20 Points) Map is displayed of Baltimore, one or more interactive elements are present.  Displayed information is non-trivial and reveals something interesting about the data.
  * (10 Points) Explination of the above map is reasonable and clear.  Addresses all points.


* Total Score:

### Credits

Thanks to [John P. Dickerson](http://jpdickerson.com/) for the project idea!